In [1]:
import pandas as pd

import plotly.express as px

In [2]:
df_matches = pd.read_csv("../data/processed/matches.csv")
df_matches.head(3)

,id_team_home,name_team_home,name_team_away,rodada,date,hour,day_of_week,city,stadium
0,1984,Criciúma,Juventude,1,2024-04-13,18:30,Saturday,Criciúma,Estádio Heriberto Hülse
1,1966,Sport Club Internacional,Bahia,1,2024-04-13,18:30,Saturday,Porto Alegre,Estádio Beira-Rio
2,1961,Fluminense,Red Bull Bragantino,1,2024-04-13,21:00,Saturday,Rio de Janeiro,Estádio do Maracanã


## Prime-time

Vamos analisar agora quais times mais tiveram jogos como mandantes nos horários nobres.
Vamos considerar diferentes classificações para horários nobres.


In [3]:
def is_prime_time(prime_times_dict, day_of_week, hour):
    
    if day_of_week in prime_times_dict and hour in prime_times_dict[day_of_week]:
        return 1
    else:
        return 0


### Domingo às 11h e 16h/17h ou Quarta-Feira às 21h30m

In [4]:

prime_times = {
        "Sunday": ["11:00", "16:00", "17:00"],
        "Wednesday": ["21:30"]
    }

df_matches_prime_time = df_matches.copy()
df_matches_prime_time["is_prime_time_match"] = df_matches_prime_time.apply(lambda row: is_prime_time(prime_times, row.day_of_week, row.hour), axis=1)


In [5]:

df_prime_times_grouped = df_matches_prime_time.groupby(by = ["name_team_home"]).agg(cnt_matches_prime_time = ('is_prime_time_match', 'mean')).sort_values(by='cnt_matches_prime_time', ascending=False)
df_prime_times_grouped

,cnt_matches_prime_time
name_team_home,
Athletico,0.750000
Cruzeiro,0.571429
Fortaleza,0.500000
Atlético Mineiro,0.444444
Corinthians,0.375000
Vitória,0.333333
Bahia,0.333333
Sport Club Internacional,0.285714
Atlético Goianiense,0.250000


## Inferior-time

Semelhante ao que fizemos com os prime-times, vamos anlisar quais times tiveram mais jogos em horários ruins 

In [6]:
def is_inferior_time(inferior_time_dict, day_of_week, hour):
    if hour in inferior_time_dict and day_of_week in inferior_time_dict[hour]:
        return 1
    else:
        return 0

### Dias de semana às 19h00

In [7]:


inferior_times = {
        "19:00": ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"],
        #"Wednesday": ["21:30"]
    }

df_matches_inferior_time = df_matches.copy()
df_matches_inferior_time["is_inferior_time_match"] = df_matches_inferior_time.apply(lambda row: is_inferior_time(inferior_times, row.day_of_week, row.hour), axis=1)


In [8]:
df_inferior_time_grouped = df_matches_inferior_time.groupby(by = ["name_team_home"]).agg(cnt_matches_inferior_time = ('is_inferior_time_match', 'mean')).sort_values(by='cnt_matches_inferior_time', ascending=False)
df_inferior_time_grouped

,cnt_matches_inferior_time
name_team_home,
Atlético Goianiense,0.375000
Juventude,0.333333
Cruzeiro,0.285714
Botafogo,0.222222
Grêmio,0.222222
Cuiabá,0.181818
Athletico,0.125000
Vasco da Gama,0.111111
Red Bull Bragantino,0.111111


## Agregando

In [9]:
teams_league_dict = {'Athletico':'LFU',
                     'Fortaleza':'LFU',
                     'Atlético Mineiro':'Libra',
                     'Cruzeiro':'LFU',
                     'Corinthians':'LFU',
                     'Vitória':'Libra',
                     'Bahia':'Libra',
                     'Criciúma':'LFU',
                     'Flamengo':'Libra',
                     'Atlético Goianiense':'LFU',
                     'Grêmio':'Libra',
                     'Palmeiras':'Libra',
                     'Sport Club Internacional':'LFU',
                     'Vasco da Gama':'LFU',
                     'Botafogo':'LFU',
                     'Juventude':'LFU',
                     'Red Bull Bragantino':'Libra',
                     'São Paulo':'Libra',
                     'Fluminense':'LFU',
                     'Cuiabá':'LFU'}

In [10]:
df_inferior_time_grouped = df_inferior_time_grouped.reset_index()
df_prime_times_grouped = df_prime_times_grouped.reset_index()

In [12]:
df_agg = pd.merge(df_prime_times_grouped, df_inferior_time_grouped, how="left",  left_on='name_team_home', right_on='name_team_home').rename(columns = {"name_team_home":"Time Mandante", "cnt_matches_prime_time":"Proporção das partidas em horário nobre", "cnt_matches_inferior_time":"Proporção das partidas em horários ruins"})
df_agg["Liga"] = df_agg["Time Mandante"].map(teams_league_dict)
df_agg.sort_values(by="Proporção das partidas em horários ruins", ascending=False)

,Time Mandante,Proporção das partidas em horário nobre,Proporção das partidas em horários ruins,Liga
8,Atlético Goianiense,0.250000,0.375000,LFU
15,Juventude,0.111111,0.333333,LFU
1,Cruzeiro,0.571429,0.285714,LFU
10,Grêmio,0.222222,0.222222,Libra
14,Botafogo,0.111111,0.222222,LFU
19,Cuiabá,0.000000,0.181818,LFU
0,Athletico,0.750000,0.125000,LFU
6,Bahia,0.333333,0.111111,Libra
11,Vasco da Gama,0.222222,0.111111,LFU
16,Red Bull Bragantino,0.111111,0.111111,Libra


In [13]:
fig = px.box(df_agg, x = "Liga", y = "Proporção das partidas em horários ruins")
fig.show()

In [19]:
fig = px.box(df_agg, x = "Liga", y = "Proporção das partidas em horário nobre")
fig.show()

In [17]:
df_agg.groupby(by = "Liga").mean().round(4).rename(columns={"Proporção das partidas em horário nobre":"Porcentagem das partidas em horário nobre", "Proporção das partidas em horários ruins":"Porcentagem das partidas em horários ruins"})*100

<ipython-input-17-949d815e0e28>:1: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



,Porcentagem das partidas em horário nobre,Porcentagem das partidas em horários ruins
Liga,,
LFU,29.06,15.38
Libra,24.72,5.56
